# Data Encoding 

## Importing the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/german_credit_data.csv")

In [3]:
df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


# Data Cleaning

The first column of this dataframe is a copy of the index. This is already included in the dataframe, so we can remove it:

In [4]:
df = df.drop(columns = ['Unnamed: 0'], axis = 1)

What is the percentage of missing values in the data?

In [5]:
df.isnull().mean() * 100

Age                  0.0
Sex                  0.0
Job                  0.0
Housing              0.0
Saving accounts     18.3
Checking account    39.4
Credit amount        0.0
Duration             0.0
Purpose              0.0
Risk                 0.0
dtype: float64

Given the high percentage of missing values for _Checking Account_, we remove it:

In [6]:
df = df.drop(columns = ['Checking account'], axis = 1)

Filling missing values with a global constant:

In [7]:
df['Saving accounts'] = df['Saving accounts'].fillna('Missing')
# df['Checking account'] = df['Checking account'].fillna('Missing')

The feature 'sex' only has 2 values. Which value should be considered base case when binary encoding is used?

In [8]:
total = df.shape[0]

male_count = df["Sex"].value_counts()[0]
female_count = df["Sex"].value_counts()[1]

print(str(100 * male_count/total) +  "% of instances describe males while " + str(100 * female_count/total) + "% of instances describe females.")

69.0% of instances describe males while 31.0% of instances describe females.


Male will be the base case, as females are less common.

Using One-Hot-Encoding:

In [9]:
tempDf = pd.get_dummies(df.drop(columns= ['Risk'],axis=1), drop_first=False)
tempDf['Risk'] = df['Risk']

df = tempDf

Considering males as base case, encoding females = 1:

In [10]:
df = df.drop(columns = ['Sex_male'], axis = 1)

Encoding our target variable:

In [11]:
df['Risk'] = df['Risk'].replace('bad', True)
df['Risk'] = df['Risk'].replace('good', False)

In [12]:
df

,Age,Job,Credit amount,Duration,Sex_female,Housing_free,Housing_own,Housing_rent,Saving accounts_Missing,Saving accounts_little,...,Saving accounts_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,67,2,1169,6,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,False
1,22,2,5951,48,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,True
2,49,1,2096,12,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,False
3,45,2,7882,42,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,False
4,53,2,4870,24,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,1,1736,12,1,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,False
996,40,3,3857,30,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,False
997,38,2,804,12,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,False
998,23,2,1845,45,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,True


# Exporting our data

## Exporting general data

In [13]:
df.to_csv('data/encoded_dataset.csv', index = False)

In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state = 0)

In [15]:
train.to_csv('data/train.csv', index = False)
test.to_csv('data/test.csv', index = False)

This train/test set fold was used to find the model with the best score on the dataset.

## Oversampling our training data

In [16]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)

In [17]:
outputVar = 'Risk'

X_train = train.drop(columns= [outputVar],axis=1)
y_train = train[outputVar]

In [18]:
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [19]:
resampled = X_resampled.copy()
resampled['Risk'] = y_resampled

resampled

,Age,Job,Credit amount,Duration,Sex_female,Housing_free,Housing_own,Housing_rent,Saving accounts_Missing,Saving accounts_little,...,Saving accounts_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,30,2,2862,36,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,False
1,27,2,3123,24,1,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,True
2,24,3,7408,60,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,True
3,25,2,1264,15,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,True
4,24,2,1554,6,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,35,2,691,12,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,True
1112,46,2,3931,48,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,True
1113,43,2,4057,24,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,True
1114,44,3,12579,24,1,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,True


In [20]:
train['Risk'].value_counts()

False    558
True     242
Name: Risk, dtype: int64

In [21]:
resampled['Risk'].value_counts()

False    558
True     558
Name: Risk, dtype: int64

In [22]:
resampled.to_csv('data/resampled.csv', index = False)

## Sampling our Data using SMOTE

In [23]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 0)

In [24]:
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [25]:
dfSmote = X_resampled.copy()
dfSmote['Risk'] = y_resampled

dfSmote

,Age,Job,Credit amount,Duration,Sex_female,Housing_free,Housing_own,Housing_rent,Saving accounts_Missing,Saving accounts_little,...,Saving accounts_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,30,2,2862,36,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,False
1,27,2,3123,24,1,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,True
2,24,3,7408,60,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,True
3,25,2,1264,15,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,True
4,24,2,1554,6,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,35,2,691,12,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,True
1112,46,2,3931,48,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,True
1113,43,2,4057,24,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,True
1114,44,3,12579,24,1,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,True


In [26]:
dfSmote['Risk'].value_counts()

False    558
True     558
Name: Risk, dtype: int64

In [27]:
dfSmote.to_csv('data/SMOTE.csv', index = False)